In [ ]:
import pandas as pd
import re
import numpy as np
from keras.utils import np_utils


In [ ]:
## load data, I'm using a different delimiter, modified \t delimiter in original  data..loading data using \t delimiter is buggy..
## multiple lines get appended to one line.. example..if i recollect, it's at line 288 or 287 - when original data
## was loaded using \t delimiter
## even after this had to manually modify line 458 in train data..buggy line of text

df = pd.read_csv('train.tsv',delimiter='\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-',encoding='utf-8',header=None)
df_val = pd.read_csv('val.tsv',delimiter='\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-',encoding='utf-8',header=None)
df_test = pd.read_csv('test_no_id.tsv',delimiter='\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-\+\-',encoding='utf-8',header=None)


In [ ]:
# denoising data
# remove http links, emojis, numbers and punctuations

http_sub=re.compile(r"http\S+")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001f199-\U0001f918"
        u"\U0001f0cf-\U000fec2c"
        u"\u0101-\uffffe3"
        "0123456789!@#$$%^&*(){}|+_\-=\"\';:/?.,<>~`\]\["                   "]+", flags=re.UNICODE)

In [ ]:
## denoise train data
for i in range(len(df)):
    #print i
    #print "original"
    #print temp
    #print "processed"
    #print i
    #print df[1][i]
    temp = df[1][i].decode('utf-8')#.lower()#.split('\n')[0].lower() # choose only first line  and convert to lower case
    temp = emoji_pattern.sub('',temp)
    temp = http_sub.sub('',temp)
    #temp = '<S> ' + temp + ' </S>'
    df[1][i] = temp#.decode('utf-8')


In [ ]:
## denoise val data
N_val = 0 # total number of characters
for i in range(len(df_val)):
    #print i
    #print "original"
    #print temp
    #print "processed"
    #print i
    
    temp = df_val[1][i].decode('utf-8')#.lower()#.split('\n')[0].lower() # choose only first line  and convert to lower case
    temp = emoji_pattern.sub('',temp)
    temp = http_sub.sub('',temp)
    #temp = '<S> ' + temp + ' </S>'
    #print df_val[1][i]
    df_val[1][i] = temp#.decode('utf-8')
    N_val = N_val + len(df_val[1][i])

In [ ]:
## denoise test data
N_test = 0 # total number of characters
for i in range(len(df_test)):
    #print i
    #print "original"
    #print temp
    #print "processed"
    #print i
    
    temp = df_test[0][i].decode('utf-8')#.lower()#.split('\n')[0].lower() # choose only first line  and convert to lower case
    temp = emoji_pattern.sub('',temp)
    temp = http_sub.sub('',temp)
    #temp = '<S> ' + temp + ' </S>'
    df_test[0][i] = temp#.decode('utf-8')
    N_test = N_test + len(df_test[0][i])

In [ ]:
# sort train data in descending order - helpful in batching
s = df[1].str.len().sort_values(ascending = False).index
df = df.reindex(s).reset_index(drop=True)


In [ ]:
## create vocab from train data
tempdict = {}
for i in range(len(df)):
    chars = list(df[1][i])
    for j in range(len(chars)):
        if chars[j] not in tempdict:
            tempdict[chars[j]] = 1
        else:
            tempdict[chars[j]] += 1

vocab = { k: v for k, v in tempdict.iteritems() if v >= 10 }

vocab['<S>'] = len(df)
vocab['</S>'] = len(df)
vocab['EOS'] = 1 #added for padding #equated to 1 to avoid log(0) errors

In [ ]:
## create out of vocab list
oovlist = []
vocab['unk'] = 0
num_oov = 0
for k,v in tempdict.iteritems():
    if v<10:
        vocab['unk'] += v
        oovlist.append(k)
        num_oov +=1

In [ ]:
print oovlist
print "length of out of vocabulary", len(oovlist)

In [ ]:
print vocab
print "length of vocabulary", len(vocab)

In [ ]:
#out of vocabulary #without denoising it's 64%
print "Percentage out of vocabulary characters", len(oovlist)*100.0/len(tempdict), "%"

In [ ]:
# part 1b
## calculate probability of each character in train data

charprob = {}
N = sum(vocab.values()) # total number of characters
for k,v in vocab.iteritems():
    charprob[k] = v*1.0/N

In [ ]:
## function to calculate perplexity of each model
def logpofs(s,chp):
    chars = list(s)
    tempsum = 0
    for char in chars:
        if char in vocab:
            tempsum = tempsum + np.log2(chp[char])
        else:
            tempsum = tempsum + np.log2(chp['unk'])
    tempsum = tempsum + np.log2(chp['</S>'])
    return tempsum

def perplexity(df, N, chp):
    probsum = 0
    for i in range(len(df)):
        sentence = df[1][i]
        probsum = probsum + logpofs(sentence,chp)
    return 2**(-1 * probsum / N)

In [ ]:
## train perplexity
print perplexity(df,N,charprob)

In [ ]:
## val perplexity calc using train vocab probability distribution
print perplexity(df_val,N_val,charprob)

In [ ]:
# create character onehot vectors / indices for embedding layer in model
temp = np.asarray(range(len(vocab)))
onehots_vocab = np_utils.to_categorical(temp)
vocabOnehotID = {}
vocabindexID = {}
i=0
for keys in vocab.keys():
    vocabOnehotID[keys] = onehots_vocab[i]
    vocabindexID[keys] = i
    i=i+1
    
# language onehot vectors / indices for embedding layer in model
langOnehotID = {}
langindexID = {}
langs = [u'en', u'es', u'pt', u'ca', u'de', u'gl', u'it', u'fr', u'eu']
for i in range(len(langs)):
    langs[i] = langs[i].decode('utf-8')
i=0
temp = np.asarray(range(len(langs)))
onehots_lang = np_utils.to_categorical(temp)
for lang in langs:
    print lang
    langOnehotID[lang] = onehots_lang[i]
    langindexID[lang] = i
    i=i+1



In [ ]:
## function to get onehot,index representations and seqlen of sentences
def GetOnehotRep(targs, istrain):
    OnehotVecs = []
    IndexVecs = []
    seqLen = []
    for lines in targs:
        temp = []
        tempI = []
        tokens = []
        if istrain:
            tokens = [('<S>').decode('utf-8')] + list(lines) + [('</S>').decode('utf-8')] #list(targs)
        else:
            tokens = list(lines) + [('</S>').decode('utf-8')] + [('EOS').decode('utf-8')] # EOS added to predict padding character at the end 

        for token in tokens:
            try:
                temp1 = vocabOnehotID[token]
                temp2 = vocabindexID[token]
            except:
                temp1 = vocabOnehotID['unk']
                temp2 = vocabindexID['unk']
            temp.append(temp1)
            tempI.append(temp2)
        OnehotVecs.append(temp)
        IndexVecs.append(tempI)
        seqLen.append(len(tokens))
    return OnehotVecs, IndexVecs, seqLen

## function to get onehot,index representations of lang
def GetOnehotLangRep(targs):
    OnehotVecs = []
    IndexVecs = []
    for targ in targs:
        OnehotVecs.append(langOnehotID[targ])
        IndexVecs.append(langindexID[targ])
    return OnehotVecs, IndexVecs

In [ ]:
## function for batching train data
# function to add padding - EOS used as padding string
def makeEqlenbatch(temp,isonehot):
    ma = max(len(l) for l in temp)
    mi = min(len(l) for l in temp)

    if ma > mi:
        for j in range(len(temp)):
            if len(temp[j]) < max:
                if isonehot:
                    temp[j] = temp[j] + [vocabOnehotID['EOS']]*(ma-len(temp[j]))
                else:
                    temp[j] = temp[j] + [vocabindexID['EOS']]*(ma-len(temp[j]))
        return temp
    else:
        return temp

# get batches of data
def getBatches(gloveVecs,langVecs, batch_size,isonehot):
    batchList = []
    batchLangList = []
    for i in range(0,len(gloveVecs),batch_size):
        if i + batch_size > len(gloveVecs):
            remvals = len(gloveVecs) - i
            temp = gloveVecs[i:i+remvals]
            tempLang = np.reshape(langVecs[i:i+remvals],(remvals,1))
            temp = makeEqlenbatch(temp,isonehot) 
            batchList.append(temp)
            if not isonehot:
                batchLangList.append(np.repeat(tempLang,np.shape(temp)[1],axis=1))
            print np.shape(temp)
        else:
            temp = gloveVecs[i:i+batch_size]
            tempLang = np.reshape(langVecs[i:i+batch_size],(batch_size,1))
            temp = makeEqlenbatch(temp,isonehot) 
            batchList.append(temp)
            print np.shape(temp)
            if not isonehot:
                batchLangList.append(np.repeat(tempLang,np.shape(temp)[1],axis=1))

    return batchList,batchLangList

In [ ]:
## function for language and sequence batching
def getLabelBatches(targVec,batch_size):
    batchList = []
    for i in range(0,len(targVec),batch_size):
        if i + batch_size > len(targVec):
            remvals = len(targVec) - i
            temp = targVec[i:i+remvals]
            print np.shape(temp)
            batchList.append(temp)
        else:
            temp = targVec[i:i+batch_size]
            batchList.append(temp)
            print np.shape(temp)
    return batchList


In [ ]:
# traindata
trainsrcOnehot,trainsrcIndex,trainsrcseqLen = GetOnehotRep(df[1],1)
traintgtOnehot,traintgtIndex,traintgtseqLen = GetOnehotRep(df[1],0)
# langrep
trainLangOnehot,trainLangIndex  = GetOnehotLangRep(df[0])

trainCaLangOnehot, trainCaLangIndex  = GetOnehotLangRep([u'ca'] * len(df))
trainDeLangOnehot, trainDeLangIndex  = GetOnehotLangRep([u'de'] * len(df))
trainEnLangOnehot, trainEnLangIndex  = GetOnehotLangRep([u'en'] * len(df))
trainEsLangOnehot, trainEsLangIndex  = GetOnehotLangRep([u'es'] * len(df))
trainEuLangOnehot, trainEuLangIndex  = GetOnehotLangRep([u'eu'] * len(df))
trainFrLangOnehot, trainFrLangIndex  = GetOnehotLangRep([u'fr'] * len(df))
trainGlLangOnehot, trainGlLangIndex  = GetOnehotLangRep([u'gl'] * len(df))
trainItLangOnehot, trainItLangIndex  = GetOnehotLangRep([u'it'] * len(df))
trainPtLangOnehot, trainPtLangIndex  = GetOnehotLangRep([u'pt'] * len(df))

trainLangIndexList = [trainEnLangIndex,trainEsLangIndex,trainPtLangIndex,trainCaLangIndex,trainDeLangIndex,trainGlLangIndex,trainItLangIndex,trainFrLangIndex,trainEuLangIndex]

In [ ]:
# valdata
valsrcOnehot,valsrcIndex,valsrcseqLen = GetOnehotRep(df_val[1],1)
valtgtOnehot,valtgtIndex,valtgtseqLen = GetOnehotRep(df_val[1],0)
# vallangrep - create separate index list for each language
valLangOnehot,valLangIndex  = GetOnehotLangRep(df_val[0])

valCaLangOnehot, valCaLangIndex  = GetOnehotLangRep([u'ca'] * len(df_val))
valDeLangOnehot, valDeLangIndex  = GetOnehotLangRep([u'de'] * len(df_val))
valEnLangOnehot, valEnLangIndex  = GetOnehotLangRep([u'en'] * len(df_val))
valEsLangOnehot, valEsLangIndex  = GetOnehotLangRep([u'es'] * len(df_val))
valEuLangOnehot, valEuLangIndex  = GetOnehotLangRep([u'eu'] * len(df_val))
valFrLangOnehot, valFrLangIndex  = GetOnehotLangRep([u'fr'] * len(df_val))
valGlLangOnehot, valGlLangIndex  = GetOnehotLangRep([u'gl'] * len(df_val))
valItLangOnehot, valItLangIndex  = GetOnehotLangRep([u'it'] * len(df_val))
valPtLangOnehot, valPtLangIndex  = GetOnehotLangRep([u'pt'] * len(df_val))

valLangIndexList = [valEnLangIndex,valEsLangIndex,valPtLangIndex,valCaLangIndex,valDeLangIndex,valGlLangIndex,valItLangIndex,valFrLangIndex,valEuLangIndex]

In [ ]:
# testdata
testsrcOnehot,testsrcIndex,testsrcseqLen = GetOnehotRep(df_test[0],1)
testtgtOnehot,testtgtIndex,testtgtseqLen = GetOnehotRep(df_test[0],0)
# testlangrep - create separate index list for each language
testCaLangOnehot, testCaLangIndex  = GetOnehotLangRep([u'ca'] * len(df_test))
testDeLangOnehot, testDeLangIndex  = GetOnehotLangRep([u'de'] * len(df_test))
testEnLangOnehot, testEnLangIndex  = GetOnehotLangRep([u'en'] * len(df_test))
testEsLangOnehot, testEsLangIndex  = GetOnehotLangRep([u'es'] * len(df_test))
testEuLangOnehot, testEuLangIndex  = GetOnehotLangRep([u'eu'] * len(df_test))
testFrLangOnehot, testFrLangIndex  = GetOnehotLangRep([u'fr'] * len(df_test))
testGlLangOnehot, testGlLangIndex  = GetOnehotLangRep([u'gl'] * len(df_test))
testItLangOnehot, testItLangIndex  = GetOnehotLangRep([u'it'] * len(df_test))
testPtLangOnehot, testPtLangIndex  = GetOnehotLangRep([u'pt'] * len(df_test))

testLangIndexList = [testEnLangIndex,testEsLangIndex,testPtLangIndex,testCaLangIndex,testDeLangIndex,testGlLangIndex,testItLangIndex,testFrLangIndex,testEuLangIndex]

In [ ]:
langindexID

In [ ]:
# indexBatches - input characterIndices
batch_size = 128
trainsrcBatch, trainLangBatch = getBatches(trainsrcIndex, trainLangIndex, batch_size,0)

# onehotBatches - output ground truth characters
batch_size = 128
traintgtBatch, _ = getBatches(traintgtOnehot, trainLangIndex, batch_size,1)

# seqlenbatches - batches of seqlen of each sentence
trainsrcseqlenBatch = getLabelBatches(trainsrcseqLen,batch_size)
traintgtseqlenBatch = getLabelBatches(traintgtseqLen,batch_size)


In [ ]:
## Char RNN model
## keras model
import keras
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras import optimizers

vocab_input = Input(shape=(None,), dtype='int32', name='vocab_input')
lang_input = Input(shape = (None,), dtype='int32', name='lang_input')

x1 = Embedding(output_dim=15, input_dim=len(vocab), input_length=None)(vocab_input)
x2 = Embedding(output_dim=6, input_dim=len(langindexID), input_length=None)(lang_input)

x = keras.layers.concatenate([x1, x2])

lstm_out = LSTM(32,return_sequences=True)(x)

output = TimeDistributed(Dense(len(vocab), activation='softmax'))(lstm_out)


In [ ]:
# Compile Model
model = Model(inputs=[vocab_input, lang_input], outputs=[output])
adam = optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


In [ ]:
# run model for n epochs
for epochs in range(50):
    print epochs
    loss = 0
    acc = 0
    for i in range(len(trainsrcBatch)):
        inputs = [np.asarray(trainsrcBatch[i]), np.asarray(trainLangBatch[i])]
        outputs = np.asarray(traintgtBatch[i])
        loss,acc = model.train_on_batch(inputs,outputs)
    print loss
    print acc

In [ ]:
import h5py
model.save('charRNN.h5') #haven't saved cause couldnt install libhdf5

In [ ]:
## train prediction
## output - list of langIDnumbers and langIDs
preds_train = np.zeros((len(trainsrcIndex),9))
j=0
for langs in trainLangIndexList:
    print j
    for i in range(len(trainsrcIndex)):
        c = np.repeat(langs[i],len(trainsrcIndex[i])).tolist()
        output_array = model.predict([np.asarray(trainsrcIndex[i]).reshape(1,len(trainsrcIndex[i])), np.asarray(c).reshape(1,len(c))])
        predictions = output_array[0]
        p = 0
        for k in range(predictions.shape[0]):
            GTindex = traintgtIndex[i][k]
            p = p + np.log(predictions[k][GTindex])
        preds_train[i][j] = p
    j = j + 1


In [ ]:
langs = [u'en', u'es', u'pt', u'ca', u'de', u'gl', u'it', u'fr', u'eu']
pred_train_indices = np.argmax(preds_train, axis=1)
pred_train_indices.tolist()

In [ ]:
## prediction accuracy
from sklearn import metrics
print "language prediction accuracy on train data", metrics.accuracy_score(trainLangIndex, pred_train_indices)


In [ ]:
## prediction accuracy
from sklearn import metrics
print "language prediction accuracy on train data", metrics.accuracy_score(trainLangIndex, pred_train_indices)


In [ ]:
# f1 for each language of validation data
en_indexes = [i for i,x in enumerate(trainLangIndex) if x == 0]
en_actual = [ trainLangIndex[i] for i in en_indexes ]
en_pred = [ pred_train_indices[i] for i in en_indexes ]
print "train - En_f1", metrics.f1_score(en_actual, en_pred, average='micro')

es_indexes = [i for i,x in enumerate(trainLangIndex) if x == 1]
es_actual = [ trainLangIndex[i] for i in es_indexes ]
es_pred = [ pred_train_indices[i] for i in es_indexes ]
print "train - Es_f1", metrics.f1_score(es_actual, es_pred, average='micro')

pt_indexes = [i for i,x in enumerate(trainLangIndex) if x == 2]
pt_actual = [ trainLangIndex[i] for i in pt_indexes ]
pt_pred = [ pred_train_indices[i] for i in pt_indexes ]
print "train - Pt_f1", metrics.f1_score(pt_actual, pt_pred, average='micro')

ca_indexes = [i for i,x in enumerate(trainLangIndex) if x == 3]
ca_actual = [ trainLangIndex[i] for i in ca_indexes ]
ca_pred = [ pred_train_indices[i] for i in ca_indexes ]
print "train - Ca_f1", metrics.f1_score(ca_actual, ca_pred, average='micro')

de_indexes = [i for i,x in enumerate(trainLangIndex) if x == 4]
de_actual = [ trainLangIndex[i] for i in de_indexes ]
de_pred = [ pred_train_indices[i] for i in de_indexes ]
print "train - train - De_f1", metrics.f1_score(de_actual, de_pred, average='micro')

gl_indexes = [i for i,x in enumerate(trainLangIndex) if x == 5]
gl_actual = [ trainLangIndex[i] for i in gl_indexes ]
gl_pred = [ pred_train_indices[i] for i in gl_indexes ]
print "train - Gl_f1", metrics.f1_score(gl_actual, gl_pred, average='micro')

it_indexes = [i for i,x in enumerate(trainLangIndex) if x == 6]
it_actual = [ trainLangIndex[i] for i in it_indexes ]
it_pred = [ pred_train_indices[i] for i in it_indexes ]
print "train - It_f1", metrics.f1_score(it_actual, it_pred, average='micro')

fr_indexes = [i for i,x in enumerate(trainLangIndex) if x == 7]
fr_actual = [ trainLangIndex[i] for i in fr_indexes ]
fr_pred = [ pred_train_indices[i] for i in fr_indexes ]
print "train - Fr_f1", metrics.f1_score(fr_actual, fr_pred, average='micro')

eu_indexes = [i for i,x in enumerate(trainLangIndex) if x == 8]
eu_actual = [ trainLangIndex[i] for i in eu_indexes ]
eu_pred = [ pred_train_indices[i] for i in eu_indexes ]
print "train - Eu_f1", metrics.f1_score(eu_actual, eu_pred, average='micro')


In [ ]:
## val prediction
## output list of langIDnumbers and langIDs
preds_val = np.zeros((len(valsrcIndex),9))
j=0
for langs in valLangIndexList:
    print j
    for i in range(len(valsrcIndex)):
        c = np.repeat(langs[i],len(valsrcIndex[i])).tolist()
        output_array = model.predict([np.asarray(valsrcIndex[i]).reshape(1,len(valsrcIndex[i])), np.asarray(c).reshape(1,len(c))])
        predictions = output_array[0]
        #print np.max(predictions[0])
        #cc
        p = 0
        for k in range(predictions.shape[0]):
            GTindex = valtgtIndex[i][k]
            p = p + np.log(predictions[k][GTindex])
        preds_val[i][j] = p
    j = j + 1


In [ ]:
langs = [u'en', u'es', u'pt', u'ca', u'de', u'gl', u'it', u'fr', u'eu']
pred_val_indices = np.argmax(preds_val, axis=1)
pred_val_indices = pred_val_indices.tolist()

In [ ]:
from sklearn import metrics
print "language prediction accuracy on val data", metrics.accuracy_score(valLangIndex, pred_val_indices)


In [ ]:
# f1 for each language of validation data
en_indexes = [i for i,x in enumerate(valLangIndex) if x == 0]
en_actual = [ valLangIndex[i] for i in en_indexes ]
en_pred = [ pred_val_indices[i] for i in en_indexes ]
print "val - En_f1", metrics.f1_score(en_actual, en_pred, average='micro')

es_indexes = [i for i,x in enumerate(valLangIndex) if x == 1]
es_actual = [ valLangIndex[i] for i in es_indexes ]
es_pred = [ pred_val_indices[i] for i in es_indexes ]
print "val - Es_f1", metrics.f1_score(es_actual, es_pred, average='micro')

pt_indexes = [i for i,x in enumerate(valLangIndex) if x == 2]
pt_actual = [ valLangIndex[i] for i in pt_indexes ]
pt_pred = [ pred_val_indices[i] for i in pt_indexes ]
print "val - Pt_f1", metrics.f1_score(pt_actual, pt_pred, average='micro')

ca_indexes = [i for i,x in enumerate(valLangIndex) if x == 3]
ca_actual = [ valLangIndex[i] for i in ca_indexes ]
ca_pred = [ pred_val_indices[i] for i in ca_indexes ]
print "val - Ca_f1", metrics.f1_score(ca_actual, ca_pred, average='micro')

de_indexes = [i for i,x in enumerate(valLangIndex) if x == 4]
de_actual = [ valLangIndex[i] for i in de_indexes ]
de_pred = [ pred_val_indices[i] for i in de_indexes ]
print "val - De_f1", metrics.f1_score(de_actual, de_pred, average='micro')

gl_indexes = [i for i,x in enumerate(valLangIndex) if x == 5]
gl_actual = [ valLangIndex[i] for i in gl_indexes ]
gl_pred = [ pred_val_indices[i] for i in gl_indexes ]
print "val - Gl_f1", metrics.f1_score(gl_actual, gl_pred, average='micro')

it_indexes = [i for i,x in enumerate(valLangIndex) if x == 6]
it_actual = [ valLangIndex[i] for i in it_indexes ]
it_pred = [ pred_val_indices[i] for i in it_indexes ]
print "val - It_f1", metrics.f1_score(it_actual, it_pred, average='micro')

fr_indexes = [i for i,x in enumerate(valLangIndex) if x == 7]
fr_actual = [ valLangIndex[i] for i in fr_indexes ]
fr_pred = [ pred_val_indices[i] for i in fr_indexes ]
print "val - Fr_f1", metrics.f1_score(fr_actual, fr_pred, average='micro')

eu_indexes = [i for i,x in enumerate(valLangIndex) if x == 8]
eu_actual = [ valLangIndex[i] for i in eu_indexes ]
eu_pred = [ pred_val_indices[i] for i in eu_indexes ]
print "val - Eu_f1", metrics.f1_score(eu_actual, eu_pred, average='micro')


In [ ]:
# perplexity of validation
# gen validation output as chars using argmax method
outputs_val = []
for i in range(len(valsrcIndex)):
    c = np.repeat(valLangIndex[i],len(valsrcIndex[i])).tolist()
    output_array = model.predict([np.asarray(valsrcIndex[i]).reshape(1,len(valsrcIndex[i])), np.asarray(c).reshape(1,len(c))])
    predictions = output_array[0]
    temp = []
    for j in range(len(predictions)):
        argm = np.argmax(predictions[j])
        temp = temp + [vocabindexID.keys()[vocabindexID.values().index(int(argm))]]
    outputs_val.append(temp)


In [ ]:
# get valperplexity based on train data character distribution
def logpofs_val(chars):
    tempsum = 0
    for char in chars:
        if char in vocab:
            tempsum = tempsum + np.log2(charprob[char])
        else:
            tempsum = tempsum + np.log2(charprob['unk'])
    tempsum = tempsum #+ np.log2(charprob['</S>']) #no need to add probability of /S because it's present in the predicted sentence
    return tempsum

def perplexity_val(df, N):
    probsum = 0
    for i in range(len(df)):
        sentence = df[i]
        probsum = probsum + logpofs_val(sentence)
    return 2**(-1 * probsum / N)

In [ ]:
print "Perplexity of predicted val data", perplexity_val(outputs_val,N_val)

In [ ]:
# test prediction
## output list of langIDnumbers and langIDs
preds_test = np.zeros((len(testsrcIndex),9))
j=0
for testLang in testLangIndexList:
    print j
    for i in range(len(testsrcIndex)):
        c = np.repeat(testLang[i],len(testsrcIndex[i])).tolist()
        output_array = model.predict([np.asarray(testsrcIndex[i]).reshape(1,len(testsrcIndex[i])), np.asarray(c).reshape(1,len(c))])
        predictions = output_array[0]
        p = 0
        for k in range(predictions.shape[0]):
            GTindex = testtgtIndex[i][k]
            p = p + np.log(predictions[k][GTindex])
        preds_test[i][j] = p
    j = j + 1


In [ ]:
langs = [u'en', u'es', u'pt', u'ca', u'de', u'gl', u'it', u'fr', u'eu']
pred_test_indices = np.argmax(preds_test, axis=1)
pred_test_indices = pred_test_indices.tolist()

In [ ]:
# write test predictions to text file
langs = [u'en', u'es', u'pt', u'ca', u'de', u'gl', u'it', u'fr', u'eu']

langIDtowrite = []
for i in pred_test_indices:
    langIDtowrite.append(langs[i])

a = np.asarray(langIDtowrite)    
a = a.astype(str)
np.savetxt('test.ids_new',a,fmt='%s')  